# State optimization in LMG model (single parameter)

This is the source codes for the example discussed in Sec. V in Ref. [1].

The Hamiltonian of the Lipkin–Meshkov–Glick (LMG) model is
\begin{align}
H_{\mathrm{LMG}}=-\frac{\lambda}{N}(J_1^2+gJ_2^2)-hJ_3,
\end{align}

where $N$ is the number of spins of the system, $\lambda$ is the spin–spin interaction strength, $h$ is the strength of the 
external field and $g$ is the anisotropic parameter. $J_i=\frac{1}{2}\sum_{j=1}^N \sigma_i^{(j)}$ ($i=1,2,3$) is the collective spin operator with $\sigma_i^{(j)}$ the $i$th Pauli matrix for the $j$th spin. In single-parameter scenario, we take $g$ as the unknown parameter to be estimated. The states are expanded as 
$|\psi\rangle=\sum^J_{m=-J}c_m|J,m\rangle$ with $|J,m\rangle$ the Dicke state and $c_m$ a complex coefficient. Here we fixed 
$J=N/2$. In this example, the probe state is optimized for both noiseless scenario and collective dephasing noise. The dynamics under collective dephasing can be expressed as
\begin{align}
\partial_t\rho = -i[H_{\mathrm{LMG}},\rho]+\gamma \left(J_3\rho J_3-\frac{1}{2}\left\{\rho, J^2_3\right\}\right)
\end{align}

with $\gamma$ the decay rate.

In this case, all searches with different algorithms start from the coherent spin state defined by
$|\theta=\frac{\pi}{2},\phi=\frac{\pi}{2}\rangle=\exp(-\frac{\theta}{2}e^{-i\phi}J_{+}+\frac{\theta}{2}e^{i\phi}J_{-})|J,J\rangle$ with $J_{\pm}=J_1{\pm}iJ_2$.

[1] M. Zhang et al., QuanEstimation: an open-source toolkit for quantum parameter estimation,
arXiv:2205.15588.


In [1]:
from quanestimation import *
import numpy as np
from qutip import *

# dimensions of the system
N = 8
# generation of the coherent spin state
psi_css = spin_coherent(0.5*N, 0.5*np.pi, 0.5*np.pi, type="ket").full()
psi_css = psi_css.reshape(1, -1)[0]
# guessed state
psi0 = [psi_css]
# free Hamiltonian
Lambda, g, h = 1.0, 0.5, 0.1
Jx, Jy, Jz = jmat(0.5*N)
Jx, Jy, Jz = Jx.full(), Jy.full(), Jz.full()
H0 = -Lambda*(np.dot(Jx, Jx) + g*np.dot(Jy, Jy))/N - h*Jz
# derivative of the free Hamiltonian on g
dH = [-Lambda*np.dot(Jy, Jy)/N]
# dissipation
decay = [[Jz, 0.1]]
# time length for the evolution
tspan = np.linspace(0., 10., 2500)

## Choose the state optimization method

In [2]:
# state optimization algorithm: AD
AD_paras = {"Adam":False, "psi0":psi0, "max_episode":300, \
            "epsilon":0.01, "beta1":0.90, "beta2":0.99}
state = StateOpt(savefile=False, method="AD", **AD_paras)

# # state optimization algorithm: PSO
# PSO_paras = {"p_num":10, "psi0":psi0, "max_episode":[1000,100], \
#              "c0":1.0, "c1":2.0, "c2":2.0, "seed":1234}
# state = StateOpt(savefile=False, method="PSO", **PSO_paras)

# # state optimization algorithm: DE
# DE_paras = {"p_num":10, "psi0":psi0, "max_episode":1000, "c":1.0, \
#             "cr":0.5, "seed":1234}
# state = StateOpt(savefile=False, method="DE", **DE_paras)

# # state optimization algorithm: NM
# NM_paras = {"p_num":20, "psi0":psi0, "max_episode":1000, \
#             "ar":1.0, "ae":2.0, "ac":0.5, "as0":0.5, "seed":1234}
# state = StateOpt(savefile=False, method="NM", **NM_paras)

In [3]:
# input the dynamics data
state.dynamics(tspan, H0, dH, decay=decay, dyn_method="expm")

## Choose the objective function

In [ ]:
# objective function: QFI
state.QFIM()

# # objective function: CFI
# state.CFIM()